In [2]:
import torch
from torchmetrics import AUROC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os

In [4]:
current_file_dir = os.getcwd()
train_results_json="../out_evaluate_patch_repasoD/train_scores.json"
val_results_json="../out_evaluate_patch_repasoD/val_scores.json"
test_results_json="../out_evaluate_patch_repasoD/test_scores.json"

train_results_json=os.path.join(current_file_dir,train_results_json)    
val_results_json=os.path.join(current_file_dir,val_results_json)    
test_results_json=os.path.join(current_file_dir,test_results_json)    




def create_tables(train_data,clases):
    labels=[]
    preds=[]
    filenames=[]
    for d in train_data:
        truth=[]
        pred=[]
        gt=d['ground_truth']
        scores=d['scores']
        d['filename']
        for defecto in clases:
            truth.append(gt[defecto])
            pred.append(scores[defecto])
        labels.append(truth)
        preds.append(pred)
        filenames.append(d['filename'])
            
    labels=torch.tensor(labels)
    preds=torch.tensor(preds)

    return preds,labels,filenames


with open(train_results_json, "r") as f:
    train_dict = json.load(f)

with open(val_results_json, "r") as f:
    val_dict = json.load(f) 

with open(test_results_json, "r") as f:
    test_dict = json.load(f)           

train_data=train_dict['train_results']
val_data = val_dict['val_results']
test_data = test_dict['test_results']
clases=list(train_dict['train_results'][0]['ground_truth'].keys())

print(len(train_data))
print(len(val_data))
print(len(test_data))

1138
326
162


In [6]:
preds_train,labels_train,filenames_train=create_tables(train_data,clases)
preds_val,labels_val,filenames_val=create_tables(val_data,clases)
preds_test,labels_test,filenames_test=create_tables(test_data,clases)
print(len(filenames_train))
print(len(filenames_val))
print(len(filenames_test))

1138
326
162


# Cantidad de muestras de con cada defecto

In [13]:
print("==========================================")
print("  ** TEST **")
print("==========================================")

n_por_defecto=labels_test.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_test.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_test.shape[0])


print("==========================================")
print("  ** VAL **")
print("==========================================")

n_por_defecto=labels_val.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_val.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_val.shape[0])

print("\n==========================================")
print("  ** TRAIN **")
print("==========================================")

n_por_defecto=labels_train.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_train.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_train.shape[0])

  ** TEST **
cracks : 3
damage_by_pliers : 0
deformed_peduncle : 17
green : 20
hollow : 12
insect : 2
light_scars : 38
mechanical_damage : 7
oil_spots : 14
plu_stickers : 10
reddish : 7
rotten : 18
scars : 25
skin_breakdown : 21
surface : 39
N sin defectos: 28
N total: 162
  ** VAL **
cracks : 13
damage_by_pliers : 15
deformed_peduncle : 27
green : 30
hollow : 14
insect : 5
light_scars : 49
mechanical_damage : 14
oil_spots : 25
plu_stickers : 20
reddish : 10
rotten : 26
scars : 52
skin_breakdown : 30
surface : 78
N sin defectos: 80
N total: 326

  ** TRAIN **
cracks : 40
damage_by_pliers : 64
deformed_peduncle : 122
green : 98
hollow : 36
insect : 13
light_scars : 189
mechanical_damage : 50
oil_spots : 62
plu_stickers : 77
reddish : 33
rotten : 101
scars : 147
skin_breakdown : 137
surface : 234
N sin defectos: 263
N total: 1138


# Análisis AUROC por categorías

In [10]:
f_auroc=AUROC(task='multilabel',num_labels=len(clases),average='none')
res=f_auroc(preds_val,labels_val.int())
res=res.tolist()

aucs={}
for c,auc in zip(clases,res):
    aucs[c]=auc
    print(f'AUC-val({c}) : {auc:.3f}')


AUC-val(cracks) : 0.974
AUC-val(damage_by_pliers) : 0.960
AUC-val(deformed_peduncle) : 0.995
AUC-val(green) : 0.968
AUC-val(hollow) : 0.954
AUC-val(insect) : 0.985
AUC-val(light_scars) : 0.964
AUC-val(mechanical_damage) : 0.981
AUC-val(oil_spots) : 0.892
AUC-val(plu_stickers) : 1.000
AUC-val(reddish) : 0.899
AUC-val(rotten) : 0.976
AUC-val(scars) : 0.959
AUC-val(skin_breakdown) : 0.943
AUC-val(surface) : 0.972


In [12]:
f_auroc=AUROC(task='multilabel',num_labels=len(clases),average='none')
res_test=f_auroc(preds_test,labels_test.int())
res_test=res_test.tolist()

aucs={}
for c,auc_val, auc_test in zip(clases,res,res_test):
    aucs[c]=auc
    print(f'AUC-val**test({c}) : {auc:.3f}  ** {auc_test:.3f}')


AUC-val**test(cracks) : 0.972  ** 0.978
AUC-val**test(damage_by_pliers) : 0.972  ** 0.000
AUC-val**test(deformed_peduncle) : 0.972  ** 1.000
AUC-val**test(green) : 0.972  ** 0.985
AUC-val**test(hollow) : 0.972  ** 0.919
AUC-val**test(insect) : 0.972  ** 0.963
AUC-val**test(light_scars) : 0.972  ** 0.921
AUC-val**test(mechanical_damage) : 0.972  ** 0.901
AUC-val**test(oil_spots) : 0.972  ** 0.917
AUC-val**test(plu_stickers) : 0.972  ** 1.000
AUC-val**test(reddish) : 0.972  ** 1.000
AUC-val**test(rotten) : 0.972  ** 0.990
AUC-val**test(scars) : 0.972  ** 0.942
AUC-val**test(skin_breakdown) : 0.972  ** 0.932
AUC-val**test(surface) : 0.972  ** 0.973
